In [1]:
!jt -t gruvboxl -fs 12 -altp -tfs 12 -nfs 12 -cellw 85% -T -N -kl

In [1]:
import pandas as pd
import numpy as np
import os,time, glob as g
import warnings as ws
import time
ws.filterwarnings("ignore")

In [2]:
#Moved files manually, renamed manually
#Consolidation
data = os.getcwd()+"\\Data"
files = g.glob(os.path.join(data,'*.xlsx'))
files

['C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\Consolidation\\Data\\ARTG.xlsx',
 'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\Consolidation\\Data\\Eudamed.xlsx',
 'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\Consolidation\\Data\\FDA PH.xlsx',
 'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\Consolidation\\Data\\FDA TH.xlsx',
 'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\Consolidation\\Data\\FDA US.xlsx',
 'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\Consolidation\\Data\\MDA MY.xlsx',
 'C:\\Users\\AbhishekSrivastava\\Desktop\\Workflow\\Python\\Dev\\Medical Devices Registry\\MDR Scripts\\Consolidation\\Data\\MOHAP.xlsx',
 'C:\\Users\\AbhishekSrivasta

In [3]:
holder = dict()
for file in files:
    
    hold = pd.read_excel(file)
    country = file.split("\\")[-1].split(".")[0]
    print(country)
    print(f"Columns: {len(hold.columns)} {hold.columns}")
    holder[country] = hold
    print(hold.shape)
    
    
    

ARTG
Columns: 25 Index(['ARTG ID', 'Sponsor Name', 'Company', 'GMDN (Code-Text)', 'Class',
       'ARTG Category', 'Good Name', 'Therapeutic Type', 'Device Name',
       'Approval Area', 'Conditions', 'Product Type', 'Start Date',
       'Effective Date', 'Functional Description', 'Intended Purpose',
       'Variant Information', 'Specific Conditions', 'Manufacturer Address',
       'Manufacturer Country', 'Public Summary', 'Consumer/Patient Info',
       'Product Info', 'Manufacturer Name', 'Product Name'],
      dtype='object')
(62305, 25)
Eudamed
Columns: 12 Index(['Unnamed: 0', 'Company', 'Actor_ID', 'Address', 'Country',
       'Applicable Legislation', 'UDI_Code', 'Nomenclature', 'Basic_UDI',
       'Risk Class', 'Device Name', 'Device Model'],
      dtype='object')
(61063, 12)
FDA PH
Columns: 8 Index(['Unnamed: 0', 'Registration ID', 'Product Name', 'Manufacturer',
       'Country of Origin', 'Issuance Date', 'Expiry Date', 'URL'],
      dtype='object')
(17334, 8)
FDA TH
Columns

In [4]:
countries = list(holder.keys())
counts = [len(holder[x]) for x in countries]
counts

[62305, 61063, 17334, 60630, 163441, 25340, 12829, 369]

In [8]:
current_folder = os.getcwd()
mappings = current_folder+"\\Mappings"
keyword_file = pd.read_excel(g.glob(os.path.join(mappings,'IVD*.xlsx'))[0])
keyword_file = keyword_file.iloc[:,:8]
keyword_file
dict_copy=keyword_file.to_dict('records')
def apply_keyword_map(grill):
    
    print("\tBegin Keyword Map...", end =" ")
    print(f"ETA: {round(len(grill) / len(dict_copy)*3, 2)}s", end="\n")
    _s = time.time()
    final = []
    print()
    for row in dict_copy:
        keyword = row['Product Model (2)']
        if not type(keyword) == float:
            grill = grill.copy()
            tmp = grill[grill['Intended Purpose'].fillna("missing").str.contains(row['Product Model (2)'],case=False)]
            tmp = tmp.copy()
            tmp['Mapped']=row['Final Segment (2) (5)']
            tmp['Player']=row['Player Name (2)']
            final.append(tmp)
    final = pd.concat(final)
    
    #final = final[['index','Intended Purpose','Mapped','Player']]
    
    
    
    
    final.drop_duplicates(subset = 'index',keep = "first", inplace = True) 

    frames = [grill, final]
    frames = frames.copy()
    grill = pd.concat(frames)
    grill = grill.copy()
    grill.drop_duplicates(subset = 'index',keep = "last", inplace = True) 
    
    
    print(f"Grill merged: {grill.shape}")
    print(f"Finished in {round(time.time() - _s,2)}s.")
    return grill
    

In [9]:
#Analysis

sources = list(holder.keys())
collect = pd.DataFrame()
cols_to_keep =  ['Company','Device Name','Approval Area','Intended Purpose', 'Manufacturer Address', 'Manufacturer Country','index']
for source in sources:
    print(source)
    hold = holder.get(source)
    grill = hold.copy()
    grill['merge_index'] = grill.index
    
    if source == "ARTG":
            grill['index'] = grill['ARTG ID'].astype(str)

    elif source == 'Eudamed':
        grill['index'] = grill['Actor_ID']
        grill = grill.rename(columns = {'Country':'Manufacturer Country','Address':'Manufacturer Address',
                                'Nomenclature':'Intended Purpose','Applicable Legislation':'Approval Area'})


    elif source == 'FDA PH':
        grill['index'] = grill['Registration ID'].astype(str)
        grill = grill.rename(columns = {'Product Name':'Device Name','Manufacturer':'Company','Country of Origin':'Manufacturer Country'})
        grill['Manufacturer Address'] = "-"
        grill['Intended Purpose'] = grill['Device Name'] + " " + grill['Company']

        grill = apply_keyword_map(grill)
        grill['Approval Area'] = grill['Mapped']
    
    elif source == 'FDA TH':
        grill['index'] = grill['CAT_NO'].astype(str)
        grill = grill.rename(columns = {'Status':'Approval Area'})
        grill['Intended Purpose'] = grill['Device Description'] + " " + grill['Product details']
        grill['Manufacturer Address'] = "-"
        grill['Manufacturer Country'] = "-"
        grill['Intended Purpose'] = grill['Device Name'] + " " + grill['Company'] + " " + grill['Intended Purpose']
        grill = apply_keyword_map(grill)
        grill['Approval Area'] = grill['Mapped']
    
        
    elif source == 'FDA US':
        grill['index'] = grill['K_Number'].astype(str)
        grill = grill.rename(columns = {'Player':'Company','Specialty':'Approval Area'})
        grill['Intended Purpose'] = grill['Device Meta']
        grill['Manufacturer Address'] = "-"
        grill['Manufacturer Country'] = "-"
        grill['Intended Purpose'] = grill['Device Name'] + " " + grill['Company'] + " " + grill['Intended Purpose']
        

    
    
    elif source == 'MDA MY':
        #display(grill.head(1))
        grill['index'] = grill['REGISTRATION NO'].astype(str)
        grill = grill.rename(columns = {'BRAND NAME':'Device Name','ESTABLISHMENT NAME':'Company'})
        grill['Intended Purpose'] = grill['NAME OF DEVICE'] + " " + grill['INTENDED PURPOSE'] + " " + grill['Company']
        grill['Approval Area'] = " "
        grill['Manufacturer Address'] = "-"
        grill['Manufacturer Country'] = "-"
        grill = apply_keyword_map(grill)
        grill['Approval Area'] = grill['Mapped']

        #display(grill.head(10))
        
    elif source == 'PMDA':
        grill['index'] = grill.index.astype(str)
        grill = grill.rename(columns = {'Review Category':'Approval Area'})
        grill['Intended Purpose'] = grill['Term Name']
        
        grill['Manufacturer Address'] = "-"
        grill['Manufacturer Country'] = "-"
        grill['Intended Purpose'] = grill['Device Name'] + " " + grill['Company'] + " " + grill['Intended Purpose']
        grill = apply_keyword_map(grill)
        grill['Approval Area'] = grill['Mapped']

    elif source == 'MOHAP':
        grill['index'] = grill.index.astype(str)
        grill = grill.rename(columns = {'Trade Name':'Device Name','Manufacturer':'Company',
                                        'Country Of Origin':'Manufacturer Country',
                                        'Supplier Address':'Manufacturer Address', 'Classification':'Approval Area'})
        grill['Intended Purpose'] = grill['Device Name']
        grill['Intended Purpose'] = grill['Device Name'] + " " + grill['Company'] + " " + grill['Intended Purpose']
        grill = apply_keyword_map(grill)
        grill['Approval Area'] = grill['Mapped']
        
    grill = grill[cols_to_keep]
    grill['Source'] = source
    
    frame = [collect,grill]
    collect = pd.concat([collect,grill])


ARTG
Eudamed
FDA PH
	Begin Keyword Map... ETA: 39.76s

Grill merged: (17334, 14)
Finished in 32.89s.
FDA TH
	Begin Keyword Map... ETA: 139.06s

Grill merged: (35158, 16)
Finished in 175.56s.
FDA US
MDA MY
	Begin Keyword Map... ETA: 58.12s

Grill merged: (25175, 19)
Finished in 125.88s.
MOHAP
	Begin Keyword Map... ETA: 29.42s

Grill merged: (12829, 19)
Finished in 23.18s.
PMDA
	Begin Keyword Map... ETA: 0.85s

Grill merged: (369, 19)
Finished in 1.72s.


In [11]:
collect.columns

Index(['Company', 'Device Name', 'Approval Area', 'Intended Purpose',
       'Manufacturer Address', 'Manufacturer Country', 'index', 'Source'],
      dtype='object')

In [14]:
collect.rename(columns = {'Device Name':'Product','Intended Purpose':'Description'}, inplace=True)

In [15]:
import pandas as pd
import numpy as np
import os,time, glob as g
import warnings as ws
import time
ws.filterwarnings("ignore")


data = os.getcwd()+"\\Data\\Wave 2"
files = g.glob(os.path.join(data,'*.xlsx'))
files

holder = dict()
for file in files:
    
    hold = pd.read_excel(file)
    country = file.split("\\")[-1].split(".")[0]
    print(country)
    print(f"Columns: {len(hold.columns)} {hold.columns}")
    holder[country] = hold
    print(hold.shape)
    
indo = holder['df_catalog']
indo.rename(columns = {'Company':'Not_req','Brand':'Company'}, inplace=True)
indo['Description'] = indo['Company'] + " "+indo['Product']
sg = holder['SG_MDR']
sg.rename(columns = {'Model_list':'Product','Registrant_list':'Company'}, inplace=True)
sg['Description'] = sg['Device_Name'] + " " + sg['Description_list']



df_catalog
Columns: 5 Index(['Product', 'Price', 'Company', 'Brand', 'Function'], dtype='object')
(53628, 5)
SG_MDR
Columns: 10 Index(['Device_Name', 'Description_list', 'Medical_Specialty_Area',
       'Medical_Device_Class', 'Device_Registration', 'Registration_Date',
       'Device_System', 'Product_Owner', 'Registrant_list', 'Model_list'],
      dtype='object')
(77064, 10)


In [17]:
grill = [indo, sg]
collect = pd.concat([collect,indo, sg])

In [20]:
hold = collect.copy()
collect.shape

(508366, 19)

In [21]:
collect.drop_duplicates(inplace=True)

In [23]:
collect.shape

(430369, 19)

In [24]:
collect.columns

Index(['Company', 'Product', 'Approval Area', 'Description',
       'Manufacturer Address', 'Manufacturer Country', 'index', 'Source',
       'Price', 'Not_req', 'Function', 'Device_Name', 'Description_list',
       'Medical_Specialty_Area', 'Medical_Device_Class', 'Device_Registration',
       'Registration_Date', 'Device_System', 'Product_Owner'],
      dtype='object')

In [25]:
#chunk = collect['Company','Product']
X = collect[['Product', 'Description']]
y = collect['Company']


In [35]:
!pip install transformers[sentencepiece]


In [31]:
collect.head(4)

,Company,Product,Approval Area,Description,Manufacturer Address,Manufacturer Country,index,Source,Price,Not_req,Function,Device_Name,Description_list,Medical_Specialty_Area,Medical_Device_Class,Device_Registration,Registration_Date,Device_System,Product_Owner
0,Tsunami Medical SRL,"Bone-screw internal spinal fixation system, no...",Medical Devices,The Ventotene Spinal System is intended for po...,"Via Emilio Giorgi 27, Modena, 41124",Italy,397231,ARTG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Unimax Medical Systems Inc,"Antifog solution, endoscopic",Medical Devices,Anti-Fog Solution is intended to be used to pr...,"8F-2 No 127 Lane 235 Pao Chiao Road, Xindian D...",Taiwan,397292,ARTG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Medwin France SAS,Pilot TLS - Central venous catheter navigation...,Medical Devices,A portable system intended to process and disp...,"9 allée de la Vigne Grande, Les Aires, 34600",France,397310,ARTG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Elekta Solutions AB,"Radionuclide system, therapeutic, teletherapy",Medical Devices,Leksell Gamma Knife® is a teletherapy device i...,"Kungstensgatan 18 Box 7593, Stockholm, SE-103 93",Sweden,397291,ARTG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
data = collect[['Company','Product','Description']]
data_chunk = data.iloc[:50000]
data_chunk.head(1)

,Company,Product,Description
0,Tsunami Medical SRL,"Bone-screw internal spinal fixation system, no...",The Ventotene Spinal System is intended for po...


In [42]:
collect.to_excel("MDR_consoldiated_05-06-2023.xlsx")
data.to_csv("labelled_data.csv")

In [9]:
data = pd.read_csv("labelled_data.csv")
del data["Unnamed: 0"]

In [12]:
data.description.isna()


0         False
1         False
2         False
3         False
4         False
          ...  
430364    False
430365    False
430366    False
430367    False
430368    False
Name: description, Length: 430369, dtype: bool

In [17]:
cols = data.columns
for col in cols:
    data[col] = data[col].astype(str)

In [2]:
from transformers import pipeline
ner = pipeline("ner", grouped_entities=True)

No model was supplied, defaulted to dbmdz/bert-large-cased-finetuned-conll03-english and revision f2482bf (https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [18]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader

# Load the labeled dataset with company, product, and description columns
labeled_data = data.copy()

# Filter out rows with empty descriptions
labeled_data = labeled_data.dropna(subset=['description'])

# Split the labeled dataset into training and validation sets
train_data, val_data = train_test_split(labeled_data, test_size=0.2, random_state=42)

# Define the tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)

# Tokenize the input descriptions for the training and validation sets
train_tokens = tokenizer.batch_encode_plus(
    train_data['description'].tolist(),
    truncation=True,
    padding=True,
    return_tensors='pt'
)
val_tokens = tokenizer.batch_encode_plus(
    val_data['description'].tolist(),
    truncation=True,
    padding=True,
    return_tensors='pt'
)

# Convert the tokenized data to tensors
train_inputs = train_tokens['input_ids']
train_masks = train_tokens['attention_mask']
train_labels = torch.tensor(train_data[['company', 'product']].values)

val_inputs = val_tokens['input_ids']
val_masks = val_tokens['attention_mask']
val_labels = torch.tensor(val_data[['company', 'product']].values)

# Create TensorDatasets and DataLoaders
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

batch_size = 16
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Set up the optimizer and training loop
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=2e-5)

epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0
    
    for batch in train_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        optimizer.zero_grad()
        
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        
        loss.backward()
        optimizer.step()
    
    avg_train_loss = total_loss / len(train_dataloader)
    print('Epoch:', epoch+1, 'Training Loss:', avg_train_loss)
    
    model.eval()
    val_loss = 0
    predictions = []
    
    for batch in val_dataloader:
        input_ids = batch[0].to(device)
        attention_mask = batch[1].to(device)
        labels = batch[2].to(device)
        
        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        loss = outputs.loss
        val_loss += loss.item()
        
        logits = outputs.logits
        predictions.extend(logits.argmax(dim=1).tolist())
    
    avg_val_loss = val_loss / len(val_dataloader)
    accuracy = (torch.tensor(predictions) == val_labels).sum().item() / len(val_labels)
    print('Epoch:', epoch+1, 'Validation Loss:', avg_val_loss, 'Validation Accuracy:', accuracy)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [ ]:
collect = collect.fillna("-")

In [ ]:
collect.to_csv('MDR Tableau input.csv')